In [1]:
import pandas as pd
import numpy as np
import re
import datetime
import requests
import bs4

pd.set_option('display.max_rows', 50)

In [2]:
# define url
base = "https://www.boxofficemojo.com/date/"


In [3]:
# take one day example
date = "2020-02-02"
url = base + date
r = requests.get(url)
soup = bs4.BeautifulSoup(r.content, "html.parser")
movies = soup.find("div", class_="a-section imdb-scroll-table-inner").find_all("tr")

### Exploring website 

In [4]:
# Title
movies[1].find("a", class_="a-link-normal").contents[0]

'Bad Boys for Life'

In [5]:
# Daily
movies[1].find_all("td", class_="a-text-right mojo-field-type-money mojo-estimatable")[0].contents[0].replace(",","").replace("$","")

'2823509'

In [6]:
# theaters
movies[1].find_all("td", class_="a-text-right mojo-field-type-positive_integer mojo-estimatable")[0].contents[0].replace(",","")

'3705'

In [7]:
# Gross accumulated
movies[1].find_all("td", class_="a-text-right mojo-field-type-money mojo-estimatable")[2].contents[0].replace(",","").replace("$","")

'148059490'

In [8]:
# days
movies[1].find_all("td", class_="a-text-right mojo-field-type-positive_integer")[1].contents[0]

'17'

In [9]:
# Distributor
movies[1].find_all("td", class_="a-text-left mojo-field-type-release_studios")[0].find("a", class_="a-link-normal").contents[0]

'Sony Pictures Releasing'

In [10]:
# rank
movies[1].find_all("td", class_="a-text-right mojo-header-column mojo-truncate mojo-field-type-rank mojo-sort-column")[0].contents[0]

'1'

### Range of dates to extract

In [11]:
# test range 2000 - 2019
date1 = '2000-01-01'
date2 = '2019-12-31'
mydates = pd.date_range(date1, date2)
mydates_list = [date.strftime("%Y-%m-%d") for date in mydates]


### Extract data from BoxOffice Mojo

In [ ]:
base = "https://www.boxofficemojo.com/date/"

# empty dataframe
df = pd.DataFrame(columns=["date","rank","title", "daily" , "theaters", "gross", "days", "distributor"]) 
    
# loop for fetch all daily data of year 2019
for date in mydates_list:
    url = base + str(date)
    r = requests.get(url)
    soup = bs4.BeautifulSoup(r.content, "html.parser")
    movies = soup.find("div", class_="a-section imdb-scroll-table-inner").find_all("tr")
    for movie in movies[1:]:
        
        rank = str(movie.find_all("td", class_="a-text-right mojo-header-column mojo-truncate mojo-field-type-rank mojo-sort-column")[0].contents[0])
        title = str(movie.find("a", class_="a-link-normal").contents[0])
        daily = str(movie.find_all("td", class_="a-text-right mojo-field-type-money mojo-estimatable")[0].contents[0].replace(",","").replace("$",""))
        theaters = str(movie.find_all("td", class_="a-text-right mojo-field-type-positive_integer mojo-estimatable")[0].contents[0].replace(",",""))
        gross = str(movie.find_all("td", class_="a-text-right mojo-field-type-money mojo-estimatable")[2].contents[0].replace(",","").replace("$",""))
        days = str(movie.find_all("td", class_="a-text-right mojo-field-type-positive_integer")[1].contents[0])
        distributor = str("-" if movie.find_all("td", class_="a-text-left mojo-field-type-release_studios")[0].find("a", class_="a-link-normal") == None else movie.find_all("td", class_="a-text-left mojo-field-type-release_studios")[0].find("a", class_="a-link-normal").contents[0])
        
        df = df.append({"date":str(date), "rank":rank, "title":title, "daily":daily, "theaters":theaters, "gross":gross, "days":days, "distributor":distributor}, ignore_index=True)

In [ ]:
# check data
df